In [1]:
import keras
import tensorflow as tf
import numpy as np
from Bio import SeqIO 
import re

Using TensorFlow backend.


In [2]:
print(tf.__version__)
print(keras.__version__)

1.8.0
2.2.0


In [3]:
import cv2
print(cv2.__version__)

3.4.5


In [4]:
file_name = 'fantom5_human_enhancer.fa'
enhancers = []
for record in SeqIO.parse(file_name, "fasta"):
    sequence = re.sub('[0123456789]','',str(record.seq))
    if len(sequence) != 401:
      print("STOP")
      break
    enhancers.append(sequence)
    
print(len(enhancers))

32693


In [5]:
def showImage(img_array):
    cv2.imshow('image_box',img_array)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [6]:
def getMaskedImg(original_img):
    # lower mask (0-10)
    lower_red = np.array([0,50,50])
    upper_red = np.array([10,255,255])
    mask0 = cv2.inRange(original_img, lower_red, upper_red)

    # upper mask (170-180)
    lower_red = np.array([170,50,50])
    upper_red = np.array([180,255,255])
    mask1 = cv2.inRange(original_img, lower_red, upper_red)

    # join my masks
    mask = mask0 + mask1

    # set my output img to zero everywhere except my mask
    output_img = original_img.copy()
    output_img[np.where(mask==0)] = 0
    
    return output_img

In [7]:
def readImg(img_path):
    img = cv2.imread(img_path)
    return img

In [8]:
def getEdgesFromInnerImg(original_img):
    
    rows, cols, color_space = img.shape
    
    valid_rows = []
    valid_cols = []
    for i in range(rows):
        if all(original_img[i][228] != [255,255,255]):
            valid_rows.append(i)

    for j in range(cols):
        if all(original_img[149][j] != [255,255,255]):
            valid_cols.append(j)

    print("UpperLeft - row: ", min(valid_rows), "col: ", min(valid_cols))
    print("UpperRight - row: ", min(valid_rows), "col: ", max(valid_cols))
    print("LowerLeft - row: ", max(valid_rows), "col: ", min(valid_cols))
    print("LowerRight - row: ", max(valid_rows), "col: ", max(valid_cols))

In [9]:
def convertToHSV(original_img):
    hsv_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2HSV)
    return hsv_img

In [10]:
def make_image(sequence):
  image_array = np.ones((32,32,4))
  for k in range(len(sequence)):
    x,y=point_list[k]
    image_array[x][y] = mapping[sequence[k].upper()]
    
  return image_array

In [11]:
def convertToOriginalCoord(current_coord):
    uppermost_row = 35
    leftmost_col = 113

    original_row = (int)((current_coord[0] - uppermost_row) / 7)
    original_col = (int)((current_coord[1] - leftmost_col) / 7)
    
    return (original_row, original_col)

In [13]:
def getAllOriginalMaskedCoord(masked_pixel):
    
    original_coords = {}
    for pix in masked_pixels:
        org_coord = convertToOriginalCoord(pix)
        original_coords[org_coord] = 1
        
    return original_coords.keys()

In [14]:
# hilbert mapping: co-ordinates with string index
def last_2_bits(x):
    return x & 3

def hindex_to_xy(hindex, N):
    positions = [
                    [0, 0],
                    [0, 1],
                    [1, 1],
                    [1, 0]
                ]

    tmp = positions[last_2_bits(hindex)]
    hindex = hindex >> 2

    # 2. iteratively compute coords
    x = tmp[0]
    y = tmp[1]

    n = 4
    while (n <= N):

        n2 = int(n / 2)
        pos_in_small_square = last_2_bits(hindex)

        if(pos_in_small_square == 0): # lower left
            tmp = x
            x = y
            y = tmp
        elif(pos_in_small_square == 1): # upper left
            x = x
            y = y + n2
        elif(pos_in_small_square == 2): # upper right
            x = x + n2
            y = y + n2
        elif(pos_in_small_square == 3): # lower right
            tmp = y
            y = (n2 - 1) - x
            x = (n2 - 1) - tmp
            x = x + n2

        hindex = hindex >> 2
        n *= 2

    return x, y

def map_hilbert_coord_with_index(order):
    N = 2**order
    coord_index_map = {}
    
    for i in range(N*N):
        coord = hindex_to_xy(i, N)
        coord_index_map[coord] = i
        
    return coord_index_map


In [15]:
# our image dimension: 32x32; hilbert order is 5 (2^5 = 32)
coord_index_map = map_hilbert_coord_with_index(5)

In [16]:
def right_pixel(cur_coord):
    row = cur_coord[0]
    col = cur_coord[1] + 1
    if col >= MAX_LEN:
        col = MAX_LEN - 1
    
    return (row, col)

def left_pixel(cur_coord):
    row = cur_coord[0]
    col = cur_coord[1] - 1
    if col < 0:
        col = 0
    
    return (row, col)

def up_pixel(cur_coord):
    row = cur_coord[0] - 1
    col = cur_coord[1]
    if row < 0:
        row = 0
    
    return (row, col)

def down_pixel(cur_coord):
    row = cur_coord[0] + 1
    col = cur_coord[1]
    if row >= MAX_LEN:
        row = MAX_LEN - 1

    return (row, col)

In [17]:
MAX_LEN = 401
MAX_DIM = 31
def getActivatedIndices(masked_coords, coord_index_map):
    activated_indices = []
    for coord in masked_coords:
        idx = coord_index_map[coord]
        if idx >= MAX_LEN:
            
            if coord[0] >= MAX_DIM:
                coord = (coord[0] - 1, coord[1])
            if coord[1] >= MAX_DIM:
                coord = (coord[0], coord[1] - 1)
            
            # right
            idx = coord_index_map[right_pixel(coord)]
            if idx < MAX_LEN and idx not in activated_indices:
                activated_indices.append(idx)
            # left
            idx = coord_index_map[left_pixel(coord)]
            if idx < MAX_LEN and idx not in activated_indices:
                activated_indices.append(idx)
            # up
            idx = coord_index_map[up_pixel(coord)]
            if idx < MAX_LEN and idx not in activated_indices:
                activated_indices.append(idx)
            # down
            idx = coord_index_map[down_pixel(coord)]
            if idx < MAX_LEN and idx not in activated_indices:
                activated_indices.append(idx)
        else:
            if idx < MAX_LEN and idx not in activated_indices:
                activated_indices.append(idx)
        
    # print(activated_indices)
    activated_indices.sort()
    return activated_indices

In [18]:
def getMaskedPixels(masked_img):
    rows, cols, color_space = masked_img.shape
    
    masked_pixels = []
    for i in range(rows):
        for j in range(cols):
            if any(masked_img[i,j] != 0):
                masked_pixels.append([i,j])
                
    return masked_pixels

In [20]:
positive_string_count = 0
positive_dataset_string = []
for k in range(len(enhancers)):
    if 'N' not in enhancers[k]:
        positive_dataset_string.append(enhancers[k])
    else:
        print("Index for seq with N: ", k)
        positive_dataset_string.append('')
    positive_string_count+=1
    
print(positive_string_count)

Index for seq with N:  1838
Index for seq with N:  20408
32693


In [24]:
# our image dimension: 32x32; hilbert order is 5 (2^5 = 32)
MAX_LEN = 401
coord_index_map = map_hilbert_coord_with_index(5)
drawn_heatmap = 15540

activated_seq_list = []
for i in range (0, drawn_heatmap-1):
    
    print("Processing Sequence: ", i)
    img_path = "heatmaps/heatmap_pos_" + str(i) + ".png"
    
    seq = positive_dataset_string[i]
    img = readImg(img_path)
    
    hsv_img = convertToHSV(img)
    masked_img = getMaskedImg(hsv_img)
    masked_pixels = getMaskedPixels(masked_img)
    masked_coords = getAllOriginalMaskedCoord(masked_pixels)
    activated_index_list = getActivatedIndices(masked_coords, coord_index_map)
    
    prevIdx = -1
    activated_seq = ''
    for idx in activated_index_list:        

        if prevIdx != -1 and idx - prevIdx > 1:
            activated_seq += "-" + str(idx - prevIdx) + "-"

        activated_seq += seq[idx]
        prevIdx = idx

    activated_seq_list.append(activated_seq.upper())
    

Processing Sequence:  0
Processing Sequence:  1
Processing Sequence:  2
Processing Sequence:  3
Processing Sequence:  4
Processing Sequence:  5
Processing Sequence:  6
Processing Sequence:  7
Processing Sequence:  8
Processing Sequence:  9
Processing Sequence:  10
Processing Sequence:  11
Processing Sequence:  12
Processing Sequence:  13
Processing Sequence:  14
Processing Sequence:  15
Processing Sequence:  16
Processing Sequence:  17
Processing Sequence:  18
Processing Sequence:  19
Processing Sequence:  20
Processing Sequence:  21
Processing Sequence:  22
Processing Sequence:  23
Processing Sequence:  24
Processing Sequence:  25
Processing Sequence:  26
Processing Sequence:  27
Processing Sequence:  28
Processing Sequence:  29
Processing Sequence:  30
Processing Sequence:  31
Processing Sequence:  32
Processing Sequence:  33
Processing Sequence:  34
Processing Sequence:  35
Processing Sequence:  36
Processing Sequence:  37
Processing Sequence:  38
Processing Sequence:  39
Processing

Processing Sequence:  320
Processing Sequence:  321
Processing Sequence:  322
Processing Sequence:  323
Processing Sequence:  324
Processing Sequence:  325
Processing Sequence:  326
Processing Sequence:  327
Processing Sequence:  328
Processing Sequence:  329
Processing Sequence:  330
Processing Sequence:  331
Processing Sequence:  332
Processing Sequence:  333
Processing Sequence:  334
Processing Sequence:  335
Processing Sequence:  336
Processing Sequence:  337
Processing Sequence:  338
Processing Sequence:  339
Processing Sequence:  340
Processing Sequence:  341
Processing Sequence:  342
Processing Sequence:  343
Processing Sequence:  344
Processing Sequence:  345
Processing Sequence:  346
Processing Sequence:  347
Processing Sequence:  348
Processing Sequence:  349
Processing Sequence:  350
Processing Sequence:  351
Processing Sequence:  352
Processing Sequence:  353
Processing Sequence:  354
Processing Sequence:  355
Processing Sequence:  356
Processing Sequence:  357
Processing S

Processing Sequence:  636
Processing Sequence:  637
Processing Sequence:  638
Processing Sequence:  639
Processing Sequence:  640
Processing Sequence:  641
Processing Sequence:  642
Processing Sequence:  643
Processing Sequence:  644
Processing Sequence:  645
Processing Sequence:  646
Processing Sequence:  647
Processing Sequence:  648
Processing Sequence:  649
Processing Sequence:  650
Processing Sequence:  651
Processing Sequence:  652
Processing Sequence:  653
Processing Sequence:  654
Processing Sequence:  655
Processing Sequence:  656
Processing Sequence:  657
Processing Sequence:  658
Processing Sequence:  659
Processing Sequence:  660
Processing Sequence:  661
Processing Sequence:  662
Processing Sequence:  663
Processing Sequence:  664
Processing Sequence:  665
Processing Sequence:  666
Processing Sequence:  667
Processing Sequence:  668
Processing Sequence:  669
Processing Sequence:  670
Processing Sequence:  671
Processing Sequence:  672
Processing Sequence:  673
Processing S

Processing Sequence:  952
Processing Sequence:  953
Processing Sequence:  954
Processing Sequence:  955
Processing Sequence:  956
Processing Sequence:  957
Processing Sequence:  958
Processing Sequence:  959
Processing Sequence:  960
Processing Sequence:  961
Processing Sequence:  962
Processing Sequence:  963
Processing Sequence:  964
Processing Sequence:  965
Processing Sequence:  966
Processing Sequence:  967
Processing Sequence:  968
Processing Sequence:  969
Processing Sequence:  970
Processing Sequence:  971
Processing Sequence:  972
Processing Sequence:  973
Processing Sequence:  974
Processing Sequence:  975
Processing Sequence:  976
Processing Sequence:  977
Processing Sequence:  978
Processing Sequence:  979
Processing Sequence:  980
Processing Sequence:  981
Processing Sequence:  982
Processing Sequence:  983
Processing Sequence:  984
Processing Sequence:  985
Processing Sequence:  986
Processing Sequence:  987
Processing Sequence:  988
Processing Sequence:  989
Processing S

Processing Sequence:  1258
Processing Sequence:  1259
Processing Sequence:  1260
Processing Sequence:  1261
Processing Sequence:  1262
Processing Sequence:  1263
Processing Sequence:  1264
Processing Sequence:  1265
Processing Sequence:  1266
Processing Sequence:  1267
Processing Sequence:  1268
Processing Sequence:  1269
Processing Sequence:  1270
Processing Sequence:  1271
Processing Sequence:  1272
Processing Sequence:  1273
Processing Sequence:  1274
Processing Sequence:  1275
Processing Sequence:  1276
Processing Sequence:  1277
Processing Sequence:  1278
Processing Sequence:  1279
Processing Sequence:  1280
Processing Sequence:  1281
Processing Sequence:  1282
Processing Sequence:  1283
Processing Sequence:  1284
Processing Sequence:  1285
Processing Sequence:  1286
Processing Sequence:  1287
Processing Sequence:  1288
Processing Sequence:  1289
Processing Sequence:  1290
Processing Sequence:  1291
Processing Sequence:  1292
Processing Sequence:  1293
Processing Sequence:  1294
P

...

Processing Sequence:  14466
Processing Sequence:  14467
Processing Sequence:  14468
Processing Sequence:  14469
Processing Sequence:  14470
Processing Sequence:  14471
Processing Sequence:  14472
Processing Sequence:  14473
Processing Sequence:  14474
Processing Sequence:  14475
Processing Sequence:  14476
Processing Sequence:  14477
Processing Sequence:  14478
Processing Sequence:  14479
Processing Sequence:  14480
Processing Sequence:  14481
Processing Sequence:  14482
Processing Sequence:  14483
Processing Sequence:  14484
Processing Sequence:  14485
Processing Sequence:  14486
Processing Sequence:  14487
Processing Sequence:  14488
Processing Sequence:  14489
Processing Sequence:  14490
Processing Sequence:  14491
Processing Sequence:  14492
Processing Sequence:  14493
Processing Sequence:  14494
Processing Sequence:  14495
Processing Sequence:  14496
Processing Sequence:  14497
Processing Sequence:  14498
Processing Sequence:  14499
Processing Sequence:  14500
Processing Sequence:

Processing Sequence:  14759
Processing Sequence:  14760
Processing Sequence:  14761
Processing Sequence:  14762
Processing Sequence:  14763
Processing Sequence:  14764
Processing Sequence:  14765
Processing Sequence:  14766
Processing Sequence:  14767
Processing Sequence:  14768
Processing Sequence:  14769
Processing Sequence:  14770
Processing Sequence:  14771
Processing Sequence:  14772
Processing Sequence:  14773
Processing Sequence:  14774
Processing Sequence:  14775
Processing Sequence:  14776
Processing Sequence:  14777
Processing Sequence:  14778
Processing Sequence:  14779
Processing Sequence:  14780
Processing Sequence:  14781
Processing Sequence:  14782
Processing Sequence:  14783
Processing Sequence:  14784
Processing Sequence:  14785
Processing Sequence:  14786
Processing Sequence:  14787
Processing Sequence:  14788
Processing Sequence:  14789
Processing Sequence:  14790
Processing Sequence:  14791
Processing Sequence:  14792
Processing Sequence:  14793
Processing Sequence:

Processing Sequence:  15052
Processing Sequence:  15053
Processing Sequence:  15054
Processing Sequence:  15055
Processing Sequence:  15056
Processing Sequence:  15057
Processing Sequence:  15058
Processing Sequence:  15059
Processing Sequence:  15060
Processing Sequence:  15061
Processing Sequence:  15062
Processing Sequence:  15063
Processing Sequence:  15064
Processing Sequence:  15065
Processing Sequence:  15066
Processing Sequence:  15067
Processing Sequence:  15068
Processing Sequence:  15069
Processing Sequence:  15070
Processing Sequence:  15071
Processing Sequence:  15072
Processing Sequence:  15073
Processing Sequence:  15074
Processing Sequence:  15075
Processing Sequence:  15076
Processing Sequence:  15077
Processing Sequence:  15078
Processing Sequence:  15079
Processing Sequence:  15080
Processing Sequence:  15081
Processing Sequence:  15082
Processing Sequence:  15083
Processing Sequence:  15084
Processing Sequence:  15085
Processing Sequence:  15086
Processing Sequence:

Processing Sequence:  15345
Processing Sequence:  15346
Processing Sequence:  15347
Processing Sequence:  15348
Processing Sequence:  15349
Processing Sequence:  15350
Processing Sequence:  15351
Processing Sequence:  15352
Processing Sequence:  15353
Processing Sequence:  15354
Processing Sequence:  15355
Processing Sequence:  15356
Processing Sequence:  15357
Processing Sequence:  15358
Processing Sequence:  15359
Processing Sequence:  15360
Processing Sequence:  15361
Processing Sequence:  15362
Processing Sequence:  15363
Processing Sequence:  15364
Processing Sequence:  15365
Processing Sequence:  15366
Processing Sequence:  15367
Processing Sequence:  15368
Processing Sequence:  15369
Processing Sequence:  15370
Processing Sequence:  15371
Processing Sequence:  15372
Processing Sequence:  15373
Processing Sequence:  15374
Processing Sequence:  15375
Processing Sequence:  15376
Processing Sequence:  15377
Processing Sequence:  15378
Processing Sequence:  15379
Processing Sequence:

error: OpenCV(3.4.5) C:\projects\opencv-python\opencv\modules\imgproc\src\color.cpp:181: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [25]:
print(len(activated_seq_list))

15539


In [26]:
with open("activated_subsequence_list_with_gap.txt", "w") as file:
    for seq in activated_seq_list:
        file.write(seq + "\n")

In [ ]:
# *****************************************************
# NEXT:
#     commonSubstringFromActivatedStrings.ipynb
# *****************************************************